In [7]:
import customtkinter as ctk
import pandas as pd
from tkinter import ttk

# CustomTkinter の初期設定
ctk.set_appearance_mode("dark")  # "dark" も可
ctk.set_default_color_theme("blue")

class App(ctk.CTk):
    def __init__(self):
        super().__init__()

        self.title("CustomTkinter GUI")
        self.geometry("900x500")
        # ====== メイン領域 ======
        self.frame_main = ctk.CTkFrame(self)
        self.frame_main.pack(fill="both", expand=True, padx=10, pady=5)
        
        # --- 左側（コントロール） ---
        self.frame_left = ctk.CTkFrame(self.frame_main, width=250)
        self.frame_left.pack(side="left", fill="y", padx=10, pady=10)
                # --- 左側（コントロール） ---
        self.frame_right = ctk.CTkFrame(self.frame_main, width=700)
        self.frame_right.pack(side="left", fill="y", padx=10, pady=10)
        
#         # 入力ボックス1
#         self.entry1 = ctk.CTkEntry(self.frame_left, placeholder_text="入力ボックス1", width=200)
#         self.entry1.place(x=30, y=50)

#         # OKボタン1（入力ボックス1 → 2）
#         self.ok_button1 = ctk.CTkButton(self.frame_left, text="OK", command=self.copy_input, width=200)
#         self.ok_button1.place(x=30, y=90)

        # 入力ボックス2
        self.entry2 = ctk.CTkEntry(self.frame_left, placeholder_text="入力ボックス2", width=200)
        self.entry2.place(x=30, y=140)

        # DataFrame生成ボタン
        self.ok_button2 = ctk.CTkButton(self.frame_left, text="DataFrame生成", command=self.show_dataframe, width=200)
        self.ok_button2.place(x=30, y=180)

        # DataFrameコピー用ボタン
        self.copy_button = ctk.CTkButton(self.frame_left, text="DataFrameコピー", command=self.copy_dataframe, width=200)
        self.copy_button.place(x=30, y=220)

        # Treeview（DataFrame 表示エリア）
        self.tree = ttk.Treeview(self.frame_right, show="headings")
        self.tree.place(x=10, y=30, width=500, height=370)

        # ステータスバー
        self.status_label = ctk.CTkLabel(self.frame_main, text="ステータス表示", anchor="w", width=900)
        self.status_label.place(x=0, y=470)

    def copy_input(self):
        value = self.entry1.get()
        self.entry2.delete(0, "end")
        self.entry2.insert(0, value)
        self.status_label.configure(text="入力ボックス1の値をコピーしました")

    def show_dataframe(self):
        text = self.entry2.get()

        # DataFrame 作成
        self.df = pd.DataFrame({
            "入力値": [text] * 3,
            "数値": [1, 2, 3]
        })

        # Treeviewを初期化
        self.tree.delete(*self.tree.get_children())
        self.tree["columns"] = list(self.df.columns)

        for col in self.df.columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        for _, row in self.df.iterrows():
            self.tree.insert("", "end", values=list(row))

        self.status_label.configure(text="DataFrameを表示しました")

    def copy_dataframe(self):
        if hasattr(self, 'df'):
            self.df.to_clipboard(index=False)
            self.status_label.configure(text="DataFrameをクリップボードにコピーしました")
        else:
            self.status_label.configure(text="DataFrameが存在しません")

# アプリ起動
if __name__ == "__main__":
    app = App()
    app.mainloop()


In [10]:
import customtkinter as ctk
import pandas as pd
from tkinter import ttk
import win32com.client

# 初期設定
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

class App(ctk.CTk):
    def __init__(self):
        super().__init__()

        self.title("CustomTkinter GUI with Excel連携")
        self.geometry("900x500")

        # 入力ボックス1
        self.entry1 = ctk.CTkEntry(self, placeholder_text="入力ボックス1", width=200)
        self.entry1.place(x=30, y=50)

        # Excel範囲入力
        self.range_entry = ctk.CTkEntry(self, placeholder_text="Excel参照範囲（例：A1:A3）", width=200)
        self.range_entry.place(x=30, y=90)
        self.range_entry.insert(0, "A1:A3")

        # Excel範囲参照ボタン
        self.excel_btn = ctk.CTkButton(self, text="Excel範囲から取得", command=self.load_from_excel, width=200)
        self.excel_btn.place(x=30, y=130)

        # Excel選択範囲参照ボタン
        self.select_button = ctk.CTkButton(self, text="Excel選択から取得", command=self.load_selected_from_excel, width=200)
        self.select_button.place(x=30, y=170)

        # OKボタン1（コピー）
        self.ok_button1 = ctk.CTkButton(self, text="OK", command=self.copy_input, width=200)
        self.ok_button1.place(x=30, y=210)

        # 入力ボックス2
        self.entry2 = ctk.CTkEntry(self, placeholder_text="入力ボックス2", width=200)
        self.entry2.place(x=30, y=250)

        # DataFrame生成ボタン
        self.ok_button2 = ctk.CTkButton(self, text="DataFrame生成", command=self.show_dataframe, width=200)
        self.ok_button2.place(x=30, y=290)

        # DataFrameコピー用ボタン
        self.copy_button = ctk.CTkButton(self, text="DataFrameコピー", command=self.copy_dataframe, width=200)
        self.copy_button.place(x=30, y=330)

        # Treeview 表示
        self.tree = ttk.Treeview(self, show="headings")
        self.tree.place(x=250, y=30, width=620, height=370)

        # ステータスバー
        self.status_label = ctk.CTkLabel(self, text="ステータス表示", anchor="w", width=900)
        self.status_label.place(x=0, y=470)

    def copy_input(self):
        value = self.entry1.get()
        self.entry2.delete(0, "end")
        self.entry2.insert(0, value)
        self.status_label.configure(text="入力ボックス1の値をコピーしました")

    def show_dataframe(self):
        text = self.entry2.get()
        self.df = pd.DataFrame({
            "入力値": text.split(","),
            "数値": list(range(1, len(text.split(",")) + 1))
        })

        self.tree.delete(*self.tree.get_children())
        self.tree["columns"] = list(self.df.columns)

        for col in self.df.columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        for _, row in self.df.iterrows():
            self.tree.insert("", "end", values=list(row))

        self.status_label.configure(text="DataFrameを表示しました")

    def copy_dataframe(self):
        if hasattr(self, 'df'):
            self.df.to_clipboard(index=False)
            self.status_label.configure(text="DataFrameをクリップボードにコピーしました")
        else:
            self.status_label.configure(text="DataFrameが存在しません")

    def load_from_excel(self):
        range_str = self.range_entry.get().strip()
        if not range_str:
            self.status_label.configure(text="参照範囲を入力してください（例: A1:A3）")
            return

        try:
            excel = win32com.client.Dispatch("Excel.Application")
            wb = excel.ActiveWorkbook
            ws = wb.ActiveSheet

            values = []
            for row in ws.Range(range_str):
                for cell in row:
                    if cell.Value is not None:
                        values.append(str(cell.Value))

            joined = ",".join(values)
            self.entry1.delete(0, "end")
            self.entry1.insert(0, joined)
            self.status_label.configure(text=f"Excelから読み込み成功（範囲: {range_str}）")

        except Exception as e:
            self.status_label.configure(text=f"Excel読み込みエラー: {e}")

    def load_selected_from_excel(self):
        try:
            excel = win32com.client.Dispatch("Excel.Application")
            wb = excel.ActiveWorkbook
            ws = wb.ActiveSheet

            selection = ws.Application.Selection
            address = selection.Address  # 範囲アドレス（例：$B$2:$D$4）

            values = []
            for row in selection:
                for cell in row:
                    if cell.Value is not None:
                        values.append(str(cell.Value))

            joined = ",".join(values)
            self.entry1.delete(0, "end")
            self.entry1.insert(0, joined)
            
            self.range_entry.delete(0, "end")
            self.range_entry.insert(0,address)
            self.status_label.configure(text=f"Excel選択範囲({address})から読み込み完了")

        except Exception as e:
            self.status_label.configure(text=f"Excel選択読み込みエラー: {e}")


# アプリ起動
if __name__ == "__main__":
    app = App()
    app.mainloop()
